Imports

In [76]:
import openpyxl
import re
import csv
import pandas as pd
from unidecode import unidecode
import datetime
import numpy as np

Xlsx para csv

In [77]:
path = "../raw_data/clientes_erp_simulado_1000.xlsx"
wb = openpyxl.load_workbook(path)
ws = wb.active
with open("../processing_data/0csv_base.csv", "w") as file:
    writer = csv.writer(file)
    for row in ws.iter_rows(max_col=6):
        new_row = []
        for cell in row:
            new_row.append(cell.value or "None")
        writer.writerow(new_row)

Retirada de informações extras

In [78]:
df = pd.read_csv("../processing_data/0csv_base.csv")
new_df = df.copy()

new_df.drop("Observações", axis = 1, inplace = True)
new_df.drop("Extra Info", axis = 1, inplace = True)
new_df.columns
new_df.to_csv("../processing_data/1sem_extra_info.csv", index=False)

Deixar textos em letras maiuscúlas

In [79]:
df = pd.read_csv("../processing_data/1sem_extra_info.csv")
with open("../processing_data/2upperall.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    for row in df.values.tolist():
        row[2] = row[2].upper()
        row[0] = row[0].upper()
        writer.writerow(row)

Padronizar número de telefone

In [80]:
df = pd.read_csv("../processing_data/2upperall.csv")
with open("../processing_data/3telefone_padronizado.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    for row in df.values.tolist():
        number = ""
        regex_digitos = re.findall("[0-9]", row[1])
        while len(regex_digitos) >= 11:
            regex_digitos.pop(0)
        regex_parenteses = re.search("[()]", str(regex_digitos))
        if len(regex_digitos) < 11:
            if regex_parenteses == None:
                regex_digitos.insert(0, "(")
                regex_digitos.insert(3, ")")
            regex_digitos.insert(4, "9")
        for num in regex_digitos:
            number += num
        row[1] = number
        writer.writerow(row)

Remover duplicatas

In [81]:
df = pd.read_csv("../processing_data/3telefone_padronizado.csv")
df.drop_duplicates(inplace=True)
with open("../processing_data/4sem_duplicatas.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    writer.writerows(df.values.tolist())

Remover Acentos

In [82]:
df = pd.read_csv("../processing_data/4sem_duplicatas.csv")
with open("../processing_data/5sem_ponto_e_antes.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    for row in df.values.tolist():
        try:
            regex_ponto = re.split("[+.]", row[0])
            new_name = regex_ponto[1]
            row[0] = new_name.lstrip()
            writer.writerow(row)
        except:
            writer.writerow(row)

Remover caractéres especiais

In [83]:
df = pd.read_csv("../processing_data/5sem_ponto_e_antes.csv")
with open("../processing_data/6sem_caracter_especial.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    for row in df.values.tolist():
        regex_caractere = re.sub(r"[^a-zA-Z\s\u00C0-\u00FF]", " ", row[0])
        row[0] = unidecode(regex_caractere.lstrip())
        writer.writerow(row)

Separar nome completo em nome e sobrenome

In [84]:
df = pd.read_csv("../processing_data/6sem_caracter_especial.csv")
new_df = df.copy()
writer = csv.writer(file)
nome_value = []
sobrenome_value = []
for row in df.values.tolist():
    nome_row = []
    sobrenome_row = [] 
    sub_strings = row[0].split()
    # 3 Strings com conjunção
    if len(sub_strings) == 3 and (len(sub_strings[1]) <= 3 and len(sub_strings[1]) > 1 ):
        nome_row.append(sub_strings[0])
        sobrenome_row.append(sub_strings[1])
        sobrenome_row.append(sub_strings[2])
    # 2 Strings sem conjunção
    elif len(sub_strings) == 2:
        nome_row.append(sub_strings[0])
        sobrenome_row.append(sub_strings[1])
    # 3 Strings sem conjunção
    elif len(sub_strings) == 3 and len(sub_strings[1]) > 3:
        nome_row.append(sub_strings[0])
        nome_row.append(sub_strings[1])
        sobrenome_row.append(sub_strings[2])
    # 4 Strings com conjunção
    elif len(sub_strings) == 4:
        nome_row.append(sub_strings[0])
        nome_row.append(sub_strings[1])
        sobrenome_row.append(sub_strings[2])
        sobrenome_row.append(sub_strings[3])
    nome_value.append(" ".join(nome_row))
    sobrenome_value.append(" ".join(sobrenome_row))
new_df.insert(0, "Nome", nome_value, True)
new_df.insert(1, "Sobrenome", sobrenome_value, True)
new_df.drop("Nome completo", axis = 1, inplace = True)
new_df.to_csv("../processing_data/7separacao_nome_sobrenome.csv", index=False)

Datas padronizadas

In [85]:
df = pd.read_csv("../processing_data/7separacao_nome_sobrenome.csv")
new_df = df.copy()
# while True:
new_row = []
for row in new_df["Data Cadastro"]:
    row_temp = []
    if row:
        try:
            data = pd.to_datetime(row)
            data_formatada = data.strftime("%Y-%m-%d")
            row_temp.append(data_formatada)
        except:
            row_temp.append("invalido")
    new_row.append(" ".join(row_temp))
new_df["Data Cadastro"] = new_row
new_df.to_csv("../processing_data/8padronizando_data.csv", index=False)

/tmp/ipykernel_376/3276985773.py:9: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data = pd.to_datetime(row)


Novo campo origem arquivo

In [86]:
df = pd.read_csv("../processing_data/8padronizando_data.csv")

linha_path = pd.DataFrame([[path] + [""] * (df.shape[1] - 1) ], columns=df.columns)
df_final = pd.concat([linha_path, df], ignore_index=True)

df_final.to_csv("../processing_data/9path_origem.csv", index=False)

Nova coluna VIP, REGULAR

In [87]:
df_base = pd.read_csv("../processing_data/0csv_base.csv")

client_stat = [""]
for row in df_base.values.tolist():
    try:
        if re.search("VIP", row[4]):
            row[4] = "VIP"
        else:
            row[4] = "REGULAR"
    except:
        row[4] = "REGULAR"
    finally:
        client_stat.append(row[4])

new_df = pd.read_csv("../processing_data/9path_origem.csv")

new_df.insert(4, "Status do Cliente", client_stat, True)

new_df.to_csv("../processing_data/10coluna_vip_regular.csv", index=False)

Nova coluna UUID

In [88]:
df = pd.read_csv("../processing_data/10coluna_vip_regular.csv")

hash_cliente = [""]

for row in df.values.tolist():
    if type(row[2]) != float:
        hash_cliente.append(abs(hash(row[2])))

new_df = df.copy()

new_df.insert(0, "Hash Cliente", hash_cliente, True)

new_df.to_csv("../processing_data/11hash_cliente.csv", index=False)

Gerar log de rejeição

In [ ]:
df = pd.read_csv("../processing_data/11hash_cliente.csv")

rejeitadas = df[df.iloc[:, -1] == "invalido"]

new_df = df[df.iloc[:, -1] != "invalido"]

rejeitadas.to_csv("../processing_data/12log_rejeicao.csv", index=False)
new_df.to_csv("../processing_data/12log_aceitos.csv", index=False)